In [1]:
import numpy as np
from scipy.stats import norm

def BlackScholes(r, S, K, T, sigma, type="c"):
    d1 = (np.log(S/K) + (r + sigma**2/2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    if type == "c":
        price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif type == "p":
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    else:
        return None
    
    return price

def binomial_tree(K, S0, sigma, T, N, r, option_type='C'):
    dt = T / N
    u = np.exp(sigma * np.sqrt(dt))
    d = np.exp(-sigma * np.sqrt(dt))
    p = (np.exp(r * dt) - d) / (u - d)
    disc = np.exp(-r * dt)

    S = np.array([S0 * (u**j) * (d**(N-j)) for j in range(N+1)])

    if option_type == 'C':
        O = np.maximum(S - K, 0)
    else:
        O = np.maximum(K - S, 0)

    for i in range(N-1, -1, -1):
        O = disc * (p * O[1:i+2] + (1-p) * O[:i+1])

    return O[0]

def compare_models(S0, K, T, sigma, r, N=100, option_type='C'):
    bs_price = BlackScholes(r, S0, K, T, sigma, type=option_type.lower())
    bt_price = binomial_tree(K, S0, sigma, T, N, r, option_type)

    print(f"Black-Scholes Price: {bs_price:.4f}")
    print(f"Binomial Tree Price ({N} steps): {bt_price:.4f}")

compare_models(S0=100, K=100, T=1, sigma=0.2, r=0.05, N=100, option_type='C')


Black-Scholes Price: 10.4506
Binomial Tree Price (100 steps): 10.4306


In [2]:
compare_models(S0=100, K=100, T=1, sigma=0.2, r=0.05, N=1000, option_type='C')

Black-Scholes Price: 10.4506
Binomial Tree Price (1000 steps): 10.4486


In [3]:
compare_models(S0=100, K=100, T=1, sigma=0.2, r=0.05, N=10000, option_type='C')

Black-Scholes Price: 10.4506
Binomial Tree Price (10000 steps): 10.4504


In [4]:
compare_models(S0=100, K=100, T=1, sigma=0.2, r=0.05, N=100000, option_type='C')

Black-Scholes Price: 10.4506
Binomial Tree Price (100000 steps): 10.4506


## Conclusion

We can see that even with a large number of steps, the binomial model is significantly less efficient than the Black-Scholes-Merton model, as its convergence is weaker. For pricing European options, Black-Scholes should be used. However, the binomial model can still be useful for pricing American options, where early exercise is possible.

To improve its efficiency, repo rates or dividends should be incorporated for call options, while interest rates need to be adjusted for puts. More robust models exist, offering greater flexibility in pricing various option types, such as the Crank-Nicholson model (finite difference method).